# Import necessary libraries

In [ ]:
#importing important libraries
!pip install pmdarima
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA
from matplotlib import pyplot as plt 

# Necessary Functions

In [ ]:
# Mean Absolute Percentage Error
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.round(np.mean(np.abs((y_pred - y_true) /np.abs(y_true))) * 100,decimals=2)
# Mean Absolute Percentage Error  
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# Mean Absolute Error
def mean_absolute_error(y_true, y_pred): 
  return np.round(np.mean(np.abs((y_pred - y_true))),decimals=2)
# Mean Sqaured Error
def mean_squared_error( y_pred,y_true): 
  return np.round(np.mean((y_pred - y_true)**2),decimals=2)
# Root Mean Sqaured Error
def root_mean_squared_error(y_pred,y_true): 
  return np.round(np.sqrt(mean_squared_error(y_pred, y_true)),decimals=2)

# Read the data

Read the input hospital data and then set the index to the 'Date' column.

Aggregate the 3 German States Holiday columns for both School and Public Holidays into one single column respectively

In [ ]:
#loading and conveting time series data by setting index as date
df = pd.read_csv('ProcessedDataset.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df.index = df.Date
#School holidays Aggregation
holidays_S = pd.DataFrame({'School_holiday': df[["S_BW", "S_H","S_RP"]].max(axis=1)})
holidays_S = holidays_S[["School_holiday"]]
holidays_P = pd.DataFrame({'Public_holiday': df[["P_BW", "P_H","P_RP"]].max(axis=1)})
holidays = holidays_P[["Public_holiday"]]
holidays['School_holiday'] = holidays_S.School_holiday

#Consider only the Occupancy
df = df.drop('Date', axis=1)
df = df[['Occupancy']]

# Train Test Split for Model Preparation

In [ ]:
#divide into train and test set
train = df['20080101':'20121231']
test = df['20130101':'20130130']
#Holidays
holidays_train = holidays['20080101':'20121231']
holidays_test = holidays['20130101':'20130130']

# Plots 

Plot the Train and test data

In [ ]:
#visualizing timeseries data
fig, ax = plt.subplots(figsize=(16,7))
train.plot(ax=ax,legend=None)
test.plot(ax=ax,color='r',legend=None)
plt.axvline(x='2013-01-01',linewidth=1.5, color='black',linestyle='--')
plt.axvspan('2013-01-01','2013-01-30', facecolor='r', alpha=0.3)
plt.axvspan('2008-01-01','2012-12-31', facecolor='g', alpha=0.3)
plt.xlabel('Date')
plt.ylabel('Occupancy')

# AUTO ARIMA 
Running Auto Arima Model to find the p, d and q parameters values and predicting on the test data
Exogenous variables - School and Public Holidays of 3 German states

Best Model considered with Least AIC

## Implementation

In [ ]:
#training model
model = auto_arima(train, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
                  max_p=10, max_q=10, max_P=10, max_Q=10, seasonal=True,
                  stepwise=False, suppress_warnings=True, D=1, max_D=10,
                  error_action='ignore',approximation = False,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
model.fit(train)
#predicting values for test 
y_pred = model.predict(n_periods=test.shape[0],exogenous=holidays_test[['School_holiday','Public_holiday']])

In [ ]:
model

In [ ]:
#predict for train
in_sample_preds = model.predict_in_sample()

## Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
# #evaluating model
# print('Mean absolute percentage error: ',mean_absolute_percentage_error(test.values, y_pred))
# print('Mean absolute error: ',mean_absolute_error(test.values, y_pred))
# print('Root Mean Square error: ',root_mean_squared_error(y_pred,test.values))

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
# print('Mean absolute percentage error: ',mean_absolute_percentage_error(train.values, in_sample_preds))
# print('Mean absolute error: ',mean_absolute_error(train.values, in_sample_preds))
# print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds,train.values))

# ARIMA 
Run the model on the Identified ARIMA model with least AIC

## Implementation

In [ ]:
ar = ARIMA(maxiter=50, method='lbfgs', order=(5, 1, 0), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
#fitting model
ar.fit(train)
#predicting values for test
y_pred_ar = ar.predict(n_periods=test.shape[0],exogenous=holidays_train[['School_holiday','Public_holiday']])

In [ ]:
#predict for train
in_sample_preds_ar = ar.predict_in_sample()

## Error Metrics
Calculation of MAPE, MAE and RMSE - Test

In [ ]:
#evaluating model
print('Mean absolute percentage error: ',mean_absolute_percentage_error(test.values, y_pred_ar))
print('Mean absolute error: ',mean_absolute_error(test.values, y_pred_ar))
print('Root Mean Square error: ',root_mean_squared_error(y_pred_ar,test.values))

Calculation of MAPE, MAE and RMSE - Train

In [ ]:
print('Mean absolute percentage error: ',mean_absolute_percentage_error(train.values, in_sample_preds_ar))
print('Mean absolute error: ',mean_absolute_error(train.values, in_sample_preds_ar))
print('Root Mean Square error: ',root_mean_squared_error(in_sample_preds_ar,train.values))

# Plots 


Actual vs Predictions (Test)

In [ ]:
# A vs P plot - Test
O_df = pd.DataFrame(columns = ['Date','Occupancy'])
O_df['Date'] = test.index
O_df['Occupancy'] = y_pred_ar
train_dft = train.loc['2012-09-01':'2012-12-31']
ax = test.plot(y='Occupancy',label='Actual',legend=True)

O_df.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(6,4),ax=ax)

Actual vs Predictions (Train)

In [ ]:
# A vs P plot - Train
O_dft = pd.DataFrame(columns = ['Date','Occupancy'])
O_dft['Date'] = train.index
O_dft['Occupancy'] = in_sample_preds_ar
ax = train.plot(y='Occupancy',label='Actual',legend=True)
O_dft.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(10,4),ax=ax)

In [ ]:
# A vs P plot - Test
O_df = pd.DataFrame(columns = ['Date','Occupancy'])
O_df['Date'] = test.index
O_df['Occupancy'] = y_pred_ar
ax = test.plot(y='Occupancy',label='Actual',legend=True)
O_df.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(6,4),ax=ax)

## plotting last month fit data

In [ ]:
# O_dft = O_dft[['20120101':'20121231']]
O_dft = O_dft.loc['2012-09-01':'2012-12-31']
train_dft = train.loc['2012-09-01':'2012-12-31']
ax = train_dft.plot(y='Occupancy',label='Actual',legend=True)
O_dft.plot(x='Date',y='Occupancy',label='Predictions',legend=True,figsize=(10,4),ax=ax)

In [ ]:
O_dft.index = O_dft['Date']

# Model Summary

In [ ]:
O_df.index = O_df['Date']
O_df = O_df[['Occupancy']]
updated_data = np.concatenate([train, O_df])
updated_model = ar.fit(updated_data)
updated_model.summary()

# 12 Validation sets testing

In [ ]:
# Initialisation model
Train_Start = pd.to_datetime('01-01-2008',format='%d-%m-%Y')
# replace year only
Train_End = pd.to_datetime('31-12-2012',format='%d-%m-%Y')
print(Train_Start)
print(Train_End)

Test_Start = Train_End + pd.to_timedelta(1, unit='D')
Test_End= Test_Start + pd.to_timedelta(29, unit='D')
print(Test_Start)
print(Test_End)

training_time = pd.DataFrame()
mape = []
mae = []
rmse = []
Train_S =[]
Train_E =[]
Test_S =[]
Test_E =[]

for i in range(1,13):
  print('********************************')
  print(i)
  Train_S.append(Train_Start)
  Train_E.append(Train_End)
  Test_S.append(Test_Start)
  Test_E.append(Test_End)

  # Before Training
  Test_df = df.loc[Test_Start:Test_End]
  period = len(Test_df.index)

  Train_df = df.loc[Train_Start:Train_End]
  print(len(Train_df.index))

  #arima model
  model = ARIMA(maxiter=50, method='lbfgs', order=(5, 1, 0), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 1),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True,exogenous=holidays_train[['School_holiday','Public_holiday']])
  #fitting model
  model.fit(Train_df)
  #predicting values for test
  forecast = model.predict(n_periods=Test_df.shape[0],exogenous=holidays_train[['School_holiday','Public_holiday']])
  mape_eachtraining = mean_absolute_percentage_error(Test_df.values,forecast)
  rmse_eachtraining = root_mean_squared_error(forecast,Test_df.values)
  mae_eachtraining = mean_absolute_error(Test_df.values,forecast)
  print('Mean absolute percentage error: ',mape_eachtraining)
  mape.append(mape_eachtraining)
  rmse.append(rmse_eachtraining)
  mae.append(mae_eachtraining)

  # After Training
  Train_Start = Train_Start + pd.to_timedelta(30, unit='D')
  Train_End = Train_End + pd.to_timedelta(30, unit='D')
  print(Train_Start)
  print(Train_End)

  Test_Start = Test_Start + pd.to_timedelta(30, unit='D')
  Test_End= Test_End + pd.to_timedelta(30, unit='D')
  print(Test_Start)
  print(Test_End)

training_time['Train_Start']= Train_S
training_time['Train_End']= Train_E
training_time['Test_Start']= Test_S
training_time['Test_End']= Test_E
training_time['MAPE']= mape
training_time['RMSE']= rmse
training_time['MAE']= mae

In [ ]:
print('Average MAPE on 12 Validation Sets:',np.mean(training_time['MAPE']))
training_time.to_csv('Model1.csv')

In [ ]:
training_time

#Plots


## Function to generate plots

In [ ]:
from bokeh.plotting import figure, show, output_notebook

# init bokeh
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#123675', c2='#000000', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    
    return p

## Plots to compare training and model predictions on trained data

In [ ]:
# #Plots of train vs predictions
# show(plot_arima(in_sample_preds_ar,train,
#                 title="Original Series & In-sample Predictions", 
#                 c2='#FF0000', forecast_start=0))

## Plots to compare test data and model predictions on test data

In [ ]:
# # visualize new forecasts
# show(plot_arima(updated_data, updated_model.predict(n_periods=30)))